# Sample notebook of Permutation test with MMD

Statistical test is a test that we confirm if two-distribution is same or not.

As a one class of the statistical tests, Permutation test is well known.

In this notebook, we present you how to run Permutation test with MMD.

In [1]:
import sys
sys.path.append("../")
sys.path.append(".")
import numpy as np
import torch
from model_criticism_mmd import MMD
from model_criticism_mmd.backends.kernels_torch import BasicRBFKernelFunction
from model_criticism_mmd.supports.permutation_tests import PermutationTest
from model_criticism_mmd.models import TwoSampleDataSet
from model_criticism_mmd import ModelTrainerTorchBackend, MMD, TwoSampleDataSet, split_data
from model_criticism_mmd.backends import kernels_torch

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [4]:
device_obj = torch.device('cpu')

Next, we set dataset. The input type into the Permutation class is `TwoSampleDataSet`.
We can set either `numpy.ndarray` or `torch.tensor`.

In [2]:
np.random.seed(seed=1)
x = np.random.normal(3, 0.5, size=(500, 2))
y = np.random.normal(3, 0.5, size=(500, 2))

Then, we run the Permutation test.

In [7]:
init_scale = torch.tensor(np.array([0.05, 0.55]))
device_obj = torch.device(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
kernel_function = BasicRBFKernelFunction(log_sigma=0.0, device_obj=device_obj, opt_sigma=True)
mmd_estimator = MMD(kernel_function_obj=kernel_function, device_obj=device_obj, scales=init_scale)
dataset_train = TwoSampleDataSet(x, y, device_obj)

permutation_tester = PermutationTest(is_normalize=True, mmd_estimator=mmd_estimator, dataset=dataset_train)
statistics = permutation_tester.compute_statistic()
threshold = permutation_tester.compute_threshold(alpha=0.05)
p_value = permutation_tester.compute_p_value(statistics)
print(f'statistics: {statistics}, threshold: {threshold}, p-value: {p_value}')
if p_value > 0.05:
    print('Same distribution!')
else:
    print('Probably different distribution!')

100%|██████████| 1000/1000 [00:15<00:00, 65.39it/s]

statistics: 0.005706118359427581, threshold: 0.23298186958692346, p-value: 0.9
Same distribution!


## Permutation test with optimized kernels

To run the permutation test, we have to define a MMD estimator who has a designed kernel function.

In normal cases, we search the optimal kernel on the given datset (i.e. trainings, optimizations).

In [8]:
n_train = 400
x_train = x[:n_train]
y_train = y[:n_train]
x_test = x[n_train:]
y_test = y[n_train:]
dataset_val = TwoSampleDataSet(x_test, y_test, device_obj=device_obj)

In [9]:
init_scale = torch.tensor(np.array([0.05, 0.55]))
kernel_function = BasicRBFKernelFunction(log_sigma=0.0, device_obj=device_obj, opt_sigma=True)
mmd_estimator = MMD(kernel_function_obj=kernel_function, device_obj=device_obj, scales=init_scale)
trainer = ModelTrainerTorchBackend(mmd_estimator=mmd_estimator, device_obj=device_obj)
trained_obj = trainer.train(dataset_training=dataset_train, 
                            dataset_validation=dataset_val, 
                            num_epochs=500, batchsize=200)

2021-08-06 11:02:10,123 - model_criticism_mmd.logger_unit - INFO - Validation at 0. MMD^2 = 0.002083155062253317, ratio = [1.41393381] obj = [-0.34637576]
2021-08-06 11:02:10,583 - model_criticism_mmd.logger_unit - INFO -      5: [avg train] MMD^2 0.013927539551370346 obj [-1.80734081] val-MMD^2 0.027928577606519733 val-ratio [2.12399064] val-obj [-0.7532967]  elapsed: 0.0
2021-08-06 11:02:12,298 - model_criticism_mmd.logger_unit - INFO -     25: [avg train] MMD^2 0.01837778834691539 obj [-1.8433611] val-MMD^2 0.03941597958890697 val-ratio [1.93707534] val-obj [-0.66117928]  elapsed: 0.0
2021-08-06 11:02:15,251 - model_criticism_mmd.logger_unit - INFO -     50: [avg train] MMD^2 0.012747605173738989 obj [-1.76003644] val-MMD^2 0.02008537317347967 val-ratio [6.42102524] val-obj [-1.8595778]  elapsed: 0.0
2021-08-06 11:02:20,707 - model_criticism_mmd.logger_unit - INFO -    100: [avg train] MMD^2 0.013348507753305464 obj [-4.83778388] val-MMD^2 0.01999693791920378 val-ratio [199.96937919

Now, we have the trained MMD estimator.

In [10]:
trained_mmd_estimator = MMD.from_trained_parameters(trained_obj, device_obj=device_obj)

Finally, we run a permutation test. For that, we call a class named `PermutationTest`.

In [11]:
permutation_tester = PermutationTest(n_permutation_test=1000, 
                                     mmd_estimator=trained_mmd_estimator, 
                                     dataset=dataset_train, 
                                     batch_size=-1)
statistics = permutation_tester.compute_statistic()
threshold = permutation_tester.compute_threshold(alpha=0.05)
p_value = permutation_tester.compute_p_value(statistics)
print(f"MMD-statistics: {statistics}, threshold: {threshold}, p-value: {p_value}")
if p_value > 0.05:
    print('Same distribution!')
else:
    print('Probably different distribution!')

100%|██████████| 1000/1000 [00:32<00:00, 31.03it/s]

MMD-statistics: 0.004012084050638276, threshold: 0.004015354932494816, p-value: 0.13
Same distribution!
